In [58]:
import configparser
import psycopg2
import pandas as pd
from tabulate import tabulate

In [59]:
def select(cur, conn, query):
    cur.execute(query)
    result = cur.fetchall()
    column_names = [tuple([row[0] for row in cur.description])]
    result = column_names + result
    conn.commit()
    return result

In [60]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [61]:
query1 = """SELECT * FROM songplays LIMIT 5;"""
result1 = select(cur, conn, query1)
pd.DataFrame(result1[1:], columns=result1[0])

,songplay_id,start_time,user_id,level,song_id,artist_id,sessionid,location,useragent
0,38,2018-11-03 16:29:46.796,95,paid,None,None,152,"Winston-Salem, NC","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like..."
1,102,2018-11-06 11:53:05.796,88,free,None,None,206,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,166,2018-11-03 18:54:58.796,95,paid,None,None,152,"Winston-Salem, NC","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like..."
3,230,2018-11-06 20:19:42.796,97,paid,None,None,293,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
4,294,2018-11-02 11:44:08.796,44,paid,None,None,157,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...


In [62]:
query2 = """SELECT * FROM users LIMIT 5;"""
result2 = select(cur, conn, query2)
pd.DataFrame(result2[1:], columns=result2[0])

,user_id,firstname,lastname,gender,level
0,100,Adler,Barrera,M,free
1,88,Mohammad,Rodriguez,M,paid
2,57,Katherine,Gay,F,free
3,63,Ayla,Johnson,F,free
4,84,Shakira,Hunt,F,free


In [63]:
query3 = """SELECT * FROM songs LIMIT 5;"""
result3 = select(cur, conn, query3)
pd.DataFrame(result3[1:], columns=result3[0])

,song_id,title,artist_id,year,duration
0,SOPABCG12AB0181B38,Te Echo De Menos,AR8B79N1187B98FB4A,2006,208.56118
1,SOXBPEM12AB0188846,The Womb (Album Version),ARMJVLT1187B9AC8DB,1996,85.60281
2,SODXJUU12A58A80D79,Surgically Removing The Tracking Device,ARRTIN91187FB4BC6F,2007,149.86404
3,SOQBLCB12AB018D6A5,Lucious,AROJZRU1187FB508F3,2009,382.56281
4,SOTKCOM12A8C134AAB,Me siento bien,AR69GGX1187B98E1F7,2007,219.58485


In [64]:
query4 = """SELECT * FROM artists LIMIT 5;"""
result4 = select(cur, conn, query4)
pd.DataFrame(result4[1:], columns=result4[0])

,artist_id,name,location,artist_latitude,artist_longitude
0,ARVEJ9M1187FB4DC44,Les Ogres De Barback,,NaN,NaN
1,ARWILYB1187FB37DFE,Bananarama,"London, England",NaN,NaN
2,ARBIE071187B9A5AF6,Janove Ottesen,,NaN,NaN
3,ARMWIHP1187B98C7D5,Colleen Et Les Boîtes À Musique,Florida,NaN,NaN
4,ARGWFMR1187B98D0E7,Cómplices,ES,-19.59634,-40.77105


In [65]:
query5 = """SELECT * FROM times LIMIT 5;"""
result5 = select(cur, conn, query5)
pd.DataFrame(result5[1:], columns=result5[0])

,start_time,hour,day,week,month,year,weekday
0,2018-11-03 01:16:15.796,1,3,44,11,2018,6
1,2018-11-03 16:07:39.796,16,3,44,11,2018,6
2,2018-11-03 16:39:49.796,16,3,44,11,2018,6
3,2018-11-03 17:02:24.796,17,3,44,11,2018,6
4,2018-11-03 17:17:26.796,17,3,44,11,2018,6


In [66]:
query5 = """SELECT u.firstName + ' ' + u.lastName as fullName, 
            a.name as artist_name, 
            s.title as song_title, 
            t.day::VARCHAR + '-' + t.month::VARCHAR + '-' + t.year::VARCHAR + '  ' + t.hour::VARCHAR + ':00' as date
            FROM songplays sp
            JOIN users u on sp.user_id = u.user_id
            JOIN artists a on sp.artist_id = a.artist_id
            JOIN songs s on sp.song_id = s.song_id
            JOIN times t on sp.start_time = t.start_time"""
result5 = select(cur, conn, query5)
pd.DataFrame(result5[1:], columns=result5[0])

,fullname,artist_name,song_title,date
0,Adler Barrera,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,20-11-2018 15:00
1,Adler Barrera,Jagged Edge,Let's Get Married,7-11-2018 7:00
2,Adler Barrera,Jagged Edge Featuring Major Damage,Let's Get Married,7-11-2018 7:00
3,Adler Barrera,Jagged Edge featuring Run of Run DMC,Let's Get Married,7-11-2018 7:00
4,Mohammad Rodriguez,Ron Carter,I CAN'T GET STARTED,27-11-2018 20:00
5,Mohammad Rodriguez,Tomcraft,Loneliness,24-11-2018 5:00
6,Mohammad Rodriguez,Dwight Yoakam,You're The One,9-11-2018 22:00
7,Mohammad Rodriguez,Mad Flava,Wax on Tha Belt (Baby G Gets Biz),24-11-2018 4:00
8,Mohammad Rodriguez,Paramore,Emergency (Album Version),26-11-2018 13:00
9,Mohammad Rodriguez,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,23-11-2018 16:00


In [68]:
conn.close()